In [33]:
import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import os
import sys
sys.path.append(os.path.abspath("../src"))
import config

import sqlite3

from sentence_transformers import SentenceTransformer

import pickle


In [34]:
from sklearn.metrics.pairwise import cosine_similarity

In [35]:
import time

In [39]:
def query_all(sample_query):
    t = time.time()
    query_vector = model.encode(sample_query)
    print(f"Encode took {time.time()-t}s")
    t = time.time()
    [scores] = cosine_similarity(query_vector.reshape(1, -1), vectors)
    print(f"Search took {time.time()-t}s")
    
    top_scores_ids = np.argsort(scores)
    top_scores_ids = top_scores_ids[::-1][:15]
    for top_id in top_scores_ids:
        tmp = metadata.iloc[top_id]
        print("Index:", top_id)
        print("Title: ", tmp.title_pt)
        print("Author: ", tmp.author)
        print("Abstract: ", tmp.abstract_pt)
        print("Keywords: ", tmp.keywords_pt)
        print("\n##############################################################################\n")

In [6]:
def query(sample_query):
    t = time.time()
    query_vector = model.encode(sample_query)
    print(f"Encode took {time.time()-t}s")
    t = time.time()
    [scores] = cosine_similarity(query_vector.reshape(1, -1), vectors_concat)
    print(f"Search took {time.time()-t}s")
    top_scores_ids = np.argsort(scores)
    top_scores_ids = top_scores_ids[::-1][:15]
    for top_id in top_scores_ids:
        tmp = metadata.iloc[idxs[top_id]]
        print("Index:", idxs[top_id])
        print("Title: ", tmp.title_en)
        print("Author: ", tmp.author)
        print("Abstract: ", tmp.abstract_en)
        print("Keywords: ", tmp.keywords_en)
        print("\n##############################################################################\n")

In [7]:
config.DATA_DIR

'/home/decarv/projects/sse/data'

In [5]:
data = pd.read_csv(os.path.join(config.DATA_DIR, "data_pt.csv"), keep_default_na=False)
metadata = pd.read_csv(os.path.join(config.DATA_DIR, "metadata_pt.csv"), keep_default_na=False)

### Indexação

In [8]:
data.iloc[73,:]

title_pt       Análise de incerteza de um sistema automatizad...
abstract_pt    Ensaios de distribuição de água de aspersores ...
keywords_pt                                                     
Name: 73, dtype: object

In [10]:
idxs = []
for i, row in enumerate(data.itertuples()):
    for _ in range(len((row.title_en + ". " + row.abstract_en + " " + row.keywords_en).split(". "))):
        idxs.append(i)

with open(os.path.join(config.DATA_DIR, "idxs.pickle"), 'wb') as f:
    pickle.dump(idxs, f)

#### Sample Row


In [11]:
sample_row = 97610

In [22]:
row = data.iloc[sample_row,:]
sample = (row.title_pt + ". " + row.abstract_pt + " " + row.keywords_pt).split(". ")
sample

['Teste de drivers de dispositivo do kernel Linux',
 'Drivers de dispositivo são uma parte essencial do kernel do Linux',
 'Bugs nesses componentes podem comprometer a estabilidade de qualquer sistema operacional GNU/Linux',
 'Para mitigar isso, os drivers de dispositivo devem ser testados em vários cenários de caso de uso',
 'No entanto, isso nem sempre é facilmente alcançável porque os drivers de dispositivo dependem de componentes de hardware que podem operar de forma não determinística, falhar inesperadamente ou estar indisponíveis para os desenvolvedores',
 'Esta pesquisa caracteriza como os drivers de dispositivo do kernel Linux são testados',
 'Para isso, realizamos um mapeamento sistemático de literatura formal, uma revisão da literatura cinzenta e uma pesquisa com mantenedores de drivers de dispositivos do Linux',
 'Por meio desses métodos de pesquisa, podemos oferecer uma visão abrangente do estado da prática dos testes de drivers de dispositivo do kernel Linux',
 'Resumimos 

In [23]:
metadata.iloc[sample_row,:]

url            https://www.teses.usp.br/teses/disponiveis/45/...
type                                     Dissertação de Mestrado
author                                         Schmitt, Marcelo 
institute                  Instituto de Matemática e Estatística
title_pt         Teste de drivers de dispositivo do kernel Linux
abstract_pt    Drivers de dispositivo são uma parte essencial...
keywords_pt        Driver de dispositivo Linux Teste de software
Name: 97610, dtype: object

### Vetorização e Busca

In [24]:
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens', device='cuda')

In [25]:
model.encode(sample).shape

(12, 768)

#### Exemplo de Encoding

In [26]:
# batch of size 64
batch = [pre for _ in range(64)]
import time
s = time.time()
vectors = model.encode(batch)
e = time.time()
print(e - s, " segundos")

NameError: name 'pre' is not defined

## Vetorizando Todo o Input

In [27]:
def encode_row(row):
    return (row.title_pt + ". " + row.abstract_pt + " " + row.keywords_pt)

In [28]:
# encoding in batches can reduce overhead costs and speed up the process
vectors = []
batch_size = 64
batch = []
for row in tqdm(data.itertuples()):
    descriptions = []
    pre = encode_row(row)
    batch.append(pre)
    if len(batch) >= batch_size:
        vectors.append(model.encode(batch, batch_size=len(batch)))
        batch = []
if len(batch) > 0:
    vectors.append(model.encode(batch))
    batch = []

vectors = np.concatenate(vectors)

0it [00:00, ?it/s]

In [29]:
np.save(os.path.join(config.DATA_DIR, "vectors_pt.npy"), vectors, allow_pickle=False)

In [40]:
query_all("ddos attack")

Encode took 0.011524438858032227s
Search took 0.13212203979492188s
Index: 90271
Title:  Performatividades hackers
Author:  Murakami, Beatriz Garcia 
Abstract:  Na genealogia do hacker é possível perceber relações entre ideologias aparentemente contraditórias como as promovidas por criadores de vírus e as praticadas por empresários do Vale do Silício. Esta ambiguidade ideológica é transferida às mídias digitais em um modelo de produção que emprega elementos lúdicos para estimular o lucro, transformando trabalho em brincadeira. Neste contexto, expandir os limites da tecnologia implica na ampliação de mecanismos de controle. Exploramos este paradoxo através de teorias que questionam a confiança depositada nos algoritmos e de práticas artísticas que rompem com funcionalidades pré-determinadas. Os trabalhos apresentados inserem espaços de reflexão e dúvida em sistemas baseados na exatidão científica. A poética surge, portanto, na interrupção ou no desvio de funcionamentos objetivos, ainda q

In [113]:
query_all("ddos attack with machine learning")

Encode took 0.016365528106689453s
Search took 0.17243170738220215s
Index: 27488
Title:  Machine learning in complex networks
Author:  Breve, Fabricio Aparecido (Catálogo USP)
Abstract:  Complex networks is a recent and active scientific research field, which concerns large scale networks with non-trivial topological structure, such as computer networks, telecommunication networks, transport networks, social networks and biological networks. Many of these networks are naturally divided into communities or modules and, therefore, uncovering their structure is one of the main problems related to complex networks study. This problem is related with the machine learning field, which is concerned with the design and development of algorithms and techniques which allow computers to learn, or increase their performance based on experience. Some of the problems identified in traditional learning techniques include: difficulties in identifying irregular forms in the attributes space; uncovering 

## Vetorizando Sentenças

In [54]:
# encoding in batches can reduce overhead costs and speed up the process
vectors = []
batch_size = 64
batch = []
for row in tqdm(data.itertuples()):
    descriptions = []
    pre = (row.title_en + ". " + row.abstract_en + " " + row.keywords_en).split(". ")
    encoding = model.encode(pre)
    vectors.append(encoding)

vectors_concat = np.concatenate(vectors)

0it [00:00, ?it/s]

In [106]:
np.save(os.path.join(config.DATA_DIR,"vectors_sentences.npy"), vectors_concat, allow_pickle=False)

In [20]:
vectors_concat = np.load(os.path.join(config.DATA_DIR, "vectors_sentences.npy"))

In [21]:
query("ddos attack")

Encode took 0.12390255928039551s
Search took 13.152531147003174s
Index: 53791
Title:  Method for mitigating against distributed denial of service attacks using multi-agent system.
Author:  Pereira, João Paulo Aragão (Catálogo USP)
Abstract:  The quality of service offered by the Internet Service Provider (ISP) depends directly on the amount of resources available at that time. In recent decades, this quality has been affected by the frequent and intense attacks that consume these resources, such as the Distributed Denial of Service (DDoS) attacks. In order to make the ISPs network more resilient to different types of DDoS attacks, techniques have been developed against such attacks over the past few years. Aiming to contribute to the improvement of such mechanisms, this dissertation presents a reactive autonomous method for detecting and mitigating DDoS attacks using a Multi-Agent system (MAS), in networks of ISPs. The main property of the proposed method is to identify characteristic 

In [98]:
query("ddos attack with machine learning")

Encode took 0.041998863220214844s
Search took 3.450878143310547s
Index: 97594
Title:  Mitigating DDoS attacks on IoT through machine learning and network functions virtualization
Author:  Oliveira, Guilherme Werneck de (Catálogo USP)
Abstract:  The Internet of Things (IoT) has undergone a rapid popularization, reaching a wide range of application domains. As a consequence, more and more IoT devices with diverse characteristics are deployed in a variety of public and private environments, progressively becoming common objects of everyday life. On the other hand, the physical infrastructure of heterogeneous systems is complex and requires efficient and dynamic solutions for managing network performance and security, at a level that allows standardized deployment and easy replication in smart industries, buildings and cities. An approach that has been gaining ground when the intention is to respond to security threats on IoT is the Network Functions Virtualization (NFV) usage. The literat

In [99]:
query("ddos attack and networking")

Encode took 0.01407766342163086s
Search took 2.9988365173339844s
Index: 97594
Title:  Mitigating DDoS attacks on IoT through machine learning and network functions virtualization
Author:  Oliveira, Guilherme Werneck de (Catálogo USP)
Abstract:  The Internet of Things (IoT) has undergone a rapid popularization, reaching a wide range of application domains. As a consequence, more and more IoT devices with diverse characteristics are deployed in a variety of public and private environments, progressively becoming common objects of everyday life. On the other hand, the physical infrastructure of heterogeneous systems is complex and requires efficient and dynamic solutions for managing network performance and security, at a level that allows standardized deployment and easy replication in smart industries, buildings and cities. An approach that has been gaining ground when the intention is to respond to security threats on IoT is the Network Functions Virtualization (NFV) usage. The literat

In [100]:
query("linux systems and networking")

Encode took 0.018716812133789062s
Search took 2.7164721488952637s
Index: 8891
Title:  Performance evaluation and comparison between IP architecture and IP over MPLS architecture
Author:  Kakihara, Carlos Marcos (Catálogo USP)
Abstract:  This work presents a comparison and evaluation of IP and the IP over MPLS routing architectures. Initially, some basic concepts of IP and MPLS protocols and their relationship with layer 2 of OSI model are introduced. Two characteristics were analyzed: the forwarding process and traffic engineering granularity. The performance analysis was realized on Linux computers routing/forwarding IP datagramas at different types of Ethernet networks. Considering the results of the tests, a discussion about performance and suitable topologies is made for each architecture.
Keywords:  label switching mpls traffic engineering

##############################################################################

Index: 71767
Title:  Energy consumption prediction in software

In [103]:
query("Message Queuing Telemetry Transport")

Encode took 0.01730203628540039s
Search took 3.857032299041748s
Index: 81935
Title:  Scalability and performance comparison between message transport systems of multiagent platforms
Author:  Rodrigues, Henrique Donâncio Nunes (Catálogo USP)
Abstract:  This work resides in the field of multiagent systems (MAS) composed of intelligent agents that are able to use Internet communication protocols. A multiagent platform is a software or framework capable of managing multiple aspects of the agent execution and their interactions. In the recent years, many MAS platforms have been developed, all of them compliant with interoperable system development standards at different levels. Also, new programming languages have been defined and new protocols have been adopted for communication in distributed systems. These facts also influenced the multiagent community with the proposition of new platforms to support the development of multiagent systems. In addition, the adoption of agents as a paradigm

In [104]:
query("governance artificial inteligence")

Encode took 0.026017189025878906s
Search took 12.290152788162231s
Index: 21480
Title:  Short term load forecasting in eletrical areas using artificial inteligence.
Author:  Guirelli, Cleber Roberto (Catálogo USP)
Abstract:  Nowadays, with privatization of utility companies and increase in competition in the energy market, companies must increase their service quality and ensure profits. Short term load forecasting is essential for operation of power systems and can increases security and reduces generation costs. Forecasting the load demands the identification of load patterns and its relations with exogenous variables such as weather. Originally, the problem was solved using mathematics and statistics with techniques such as time series, which produces good results but are complex and have a difficult modeling. With the advent of artificial intelligence techniques, new tools capable of dealing with large amounts of data and learn by themselves system variables relations were available

In [105]:
query("how to train your dragon")

Encode took 0.04903101921081543s
Search took 9.040952444076538s
Index: 59396
Title:  Territoriality of Zenithoptera lanei (Anisoptera: Libellulidae) in an area of brazilian savana.
Author:  Valdivia, Fernando Geronimo Ancco (Catálogo USP)
Abstract:  Animal behavior associated with morphological characteristics supplies evidence for a better understanding of how sexual behavior has evolved, and how organisms can maximize their reproductive success. Factors such as size, fat reserves and other sexual characters such as coloration are essential to individual better performance in relation to conspecifics. These factors? Can bring information of the male quality to other conspecifics and females. This quality keeps relation with the physical condition (energy reserves) that they have which is required to succeed in breeding, as reproduction is one of the most expensive activity in energy cost in relation to other activities. The reproductive behavior is related to the individual fat reserv

In [23]:
query("population perceptions of dengue")

Encode took 0.016208410263061523s
Search took 4.323176622390747s
Index: 58663
Title:  Spatial patterns and climatological variables associated with dengue in the city of Ribeirão Preto between 2001 and 2010
Author:  Caminiti, Gustavo Bussi (Catálogo USP)
Abstract:  Dengue is the most important among the arboviruses, which affects man and represents a severe Public Health Problem. In Brazil, dengue is present in all 27 states of the Federation, distribution across 3,794 cities, and is responsible for about 60% of the notifications in the Americas. One of the cities in the State of São Paulo with one of the highest confirmed case ratios is Ribeirão Preto. This study aimed to characterize the spatial pattern and associate the cases of dengue with the climatological variables in the city of Ribeirão Preto between 2001 and 2010. Research with a hybrid, ecological and temporal trend design. The population included the confirmed cases of dengue among residents. The data were collected from th